# 在 GPU 上使用DeepSpeed-Inference加速GPT的Inference过程

使用Hugging Face Transformers和DeepSpeed-Inference来优化GPT-2/GPT-J的推理性能。

- 设置开发环境
- 加载初始的GPT-J模型并设定基线
- 使用DeepSpeed的InferenceEngine优化GPT-J以适配GPU
- 评估性能和速度

注意！本教程是在包含NVIDIA T4的g4dn.2xlarge AWS EC2实例上创建和运行的，请自行学习如何使用云服务器 或 云服务器大模型服务提供商。

---

## 什么是Deepspeed Inference

- DeepSpeed-Inference是DeepSpeed框架的扩展，用于提升推理工作负载。
- DeepSpeed Inference结合了张量并行（tensor parallelism）、流水线并行（pipeline-parallelism）等模型并行技术，并使用了自定义优化的CUDA内核。
- DeepSpeed为使用DeepSpeed、Megatron和HuggingFace训练的兼容Transformer模型提供了无缝的推理模式。
- 举例来说，DeepSpeed-Inference集成了模型并行技术，允许您在多GPU上运行大型语言模型（LLM）的推理，如具有1760亿参数的BLOOM。


## 1. 设置开发环境

- 安装DeepSpeed，以及PyTorch、Transformers和其他一些库。
- 运行以下代码单元格将安装所有必需的包。

_注意：需要一台带有GPU并安装了兼容的CUDA的机器。

In [ ]:
!pip install torch==1.11.0 torchvision==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113 --upgrade -q 
# !pip install deepspeed==0.7.2 --upgrade -q 
!pip install git+https://github.com/microsoft/DeepSpeed.git@ds-inference/support-large-token-length --upgrade
!pip install transformers[sentencepiece]==4.21.2 accelerate --upgrade -q 


- 在开始之前，我们需要确认所有的packages都正常安装。

In [2]:
import re  # 导入正则表达式库re，用于字符串匹配
import torch  # 导入PyTorch库

# 检查DeepSpeed的安装情况
# 使用deepspeed.env_report命令输出当前DeepSpeed环境的信息
report = !python3 -m deepspeed.env_report

# 使用正则表达式编译一个模式，用于匹配输出中的'ninja'状态是否为'OKAY'
r = re.compile('.*ninja.*OKAY.*')

# 断言判断，如果report中没有匹配到'ninja' OKAY状态，则抛出异常提示DeepSpeed Inference未正确安装
assert any(r.match(line) for line in report) == True, "DeepSpeed Inference not correctly installed"

# 检查CUDA和PyTorch版本
# 从torch.__version__获取torch和cuda版本信息
torch_version, cuda_version = torch.__version__.split("+")

# 只保留torch版本的前两位，比如从'1.9.1'中提取'1.9'
torch_version = ".".join(torch_version.split(".")[:2])

# 格式化CUDA版本为标准显示格式，例如'cu101'变为'10.1'
cuda_version = f"{cuda_version[2:4]}.{cuda_version[4:]}"

# 正则表达式用于匹配DeepSpeed报告中的torch版本信息
r = re.compile(f'.*torch.*{torch_version}.*')
# 如果版本不匹配，抛出错误提示
assert any(r.match(line) for line in report) == True, "Wrong Torch version"

# 正则表达式用于匹配DeepSpeed报告中的cuda版本信息
r = re.compile(f'.*cuda.*{cuda_version}.*')
# 如果版本不匹配，抛出错误提示
assert any(r.match(line) for line in report) == True, "Wrong Cuda version"


## 2. 加载原生 GPT-J 模型并设置baseline

- 在设置好环境后，为模型创建一个基线
- 此处使用的是EleutherAI/gpt-j-6B，一个由EleutherAI训练的GPT-J 6B模型。该模型在一个大规模的精选数据集The Pile上进行了训练
- 训练过程中，使用TPU v3-256 pod在383,500步内处理了4020亿个tokens
- 它作为一个自回归语言模型（autoregressive language model）进行训练
- 使用交叉熵损失（cross-entropy loss）来最大化正确预测下一个token的概率

- 使用transformers加载模型并运行推理，创建基线。

_注意：这里创建了一个单独的仓库，其中包含分片的fp16权重，以便通过使用device_map功能自动将分片的检查点加载到GPU上，从而更容易在较小的CPU上加载模型_

In [3]:
import torch  # 导入PyTorch库
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline  # 导入Transformers库中的必要模块

# Hugging Face模型库中的模型仓库ID
model_id = "philschmid/gpt-j-6B-fp16-sharded"

# 加载模型和分词器
# 使用AutoTokenizer加载分词器，该分词器与模型ID匹配
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 使用AutoModelForCausalLM加载因果语言模型
# 这里设置torch_dtype为float16以减少内存使用，同时使用device_map="auto"自动将所有模型分片放置到GPU上
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

# 输出确认模型已加载到设备上
print(f"model is loaded on device {model.device.type}")


model is loaded on device cuda


In [4]:
# 定义输入文本（payload），这是需要模型生成回复的内容
payload = ("Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. "
           "What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer this "
           "email in the next 7 days. Best regards and have a nice weekend but it")

# 使用分词器将输入文本转换为模型可接受的输入ID，并将其放置到与模型相同的设备上（如GPU）
input_ids = tokenizer(payload, return_tensors="pt").input_ids.to(model.device)

# 打印输入的payload内容
print(f"input payload: \n\n{payload}")

# 使用加载的模型进行文本生成推理
# 参数解释：
# - `do_sample=True`: 表示使用采样方法生成文本（而不是贪婪搜索）
# - `num_beams=1`: 表示使用单束搜索（即不进行束搜索优化）
# - `min_length=128`: 生成的最小长度为128个token
# - `max_new_tokens=128`: 最大生成128个新的token
logits = model.generate(input_ids, do_sample=True, num_beams=1, min_length=128, max_new_tokens=128)

# 打印模型生成的预测输出
# 使用`tokenizer.decode`方法将生成的token ID转回人类可读的文本
# `logits[0].tolist()[len(input_ids[0]):]`用于提取生成的部分，而不是包括输入部分
print(f"prediction: \n\n{tokenizer.decode(logits[0].tolist()[len(input_ids[0]):])}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input payload: 
 
Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer this email in the next 7 days. Best regards and have a nice weekend but it
prediction: 
 
 's Friday evening for the British and you can feel that coming in on top of a Friday, please try to spend a quiet time tonight. Thankyou, Philipp

Annette

Thank you for your reply to my last email. Regarding your issue with your new credit card please forward your request by email to "customer.service@lodging.com" In order for this to happen the email you send will need to include your full name, card number and the account number that your new card is linked to. Your credit card account number should be at the top of the email to avoid any misinterpretation of the request


In [5]:
# 测试模型生成能力
# 定义一个简单的输入示例
example = "My name is Philipp and I"

# 使用分词器将输入示例转换为模型可接受的输入ID，并将其转移到模型所在的设备（GPU）
input_ids = tokenizer(example, return_tensors="pt").input_ids.to(model.device)

# 使用加载的模型生成文本
# 参数解释：
# - `do_sample=True`: 表示使用采样策略来生成输出，增加多样性
# - `max_length=100`: 生成的文本最大长度为100个token
logits = model.generate(input_ids, do_sample=True, max_length=100)

# 将生成的token ID转换为人类可读的文本
output_text = tokenizer.decode(logits[0].tolist())

# 打印生成的文本结果
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"My name is Philipp and I'm from Germany.\nI have been a collector of music since I am a small child. I own about 4500 vinyls and 10k CDs but my collection is by no means the most important thing in my life. I am married with two kids.\nI got into the IT business many years ago, worked in the game industry where I learned a lot about programming, but now I live on the other side of the fence. We sell our house and are"

- 使用measure_latency函数来创建延迟基线，该函数通过一个简单的Python循环来运行推理，并计算模型的平均延迟（avg）、中位延迟（mean）和95百分位延迟（p95）

In [6]:
from time import perf_counter  # 从time库中导入perf_counter，用于高精度计时
import numpy as np  # 导入numpy库，用于计算延迟的统计量
import transformers  # 导入transformers库

# 隐藏生成时的警告信息
transformers.logging.set_verbosity_error()

def measure_latency(model, tokenizer, payload, generation_args={}, device=model.device):
    """
    测量模型的推理延迟。

    参数：
    - model: 已加载的语言模型
    - tokenizer: 模型对应的分词器
    - payload: 要进行推理的输入文本
    - generation_args: 生成参数的字典（默认空）
    - device: 运行推理的设备（默认为模型的设备）

    返回值：
    - 延迟统计信息的字符串格式
    - p95延迟值
    """
    # 使用分词器将输入文本转换为模型可接受的输入ID，并将其放置到指定设备上
    input_ids = tokenizer(payload, return_tensors="pt").input_ids.to(device)
    latencies = []  # 初始化一个列表来存储每次推理的延迟时间

    # 预热模型（warm up）
    for _ in range(2):
        _ = model.generate(input_ids, **generation_args)

    # 测量推理延迟
    for _ in range(10):
        start_time = perf_counter()  # 记录推理开始时间
        _ = model.generate(input_ids, **generation_args)  # 运行模型推理
        latency = perf_counter() - start_time  # 计算单次推理的延迟
        latencies.append(latency)  # 将延迟添加到列表中

    # 计算延迟的统计量
    time_avg_ms = 1000 * np.mean(latencies)  # 计算平均延迟（毫秒）
    time_std_ms = 1000 * np.std(latencies)  # 计算延迟的标准差（毫秒）
    time_p95_ms = 1000 * np.percentile(latencies, 95)  # 计算95百分位延迟（毫秒）

    # 返回格式化的延迟统计信息和95百分位延迟值
    return f"P95 latency (ms) - {time_p95_ms:.2f}; Average latency (ms) - {time_avg_ms:.2f} ± {time_std_ms:.2f};", time_p95_ms

# 示例使用
payload = "My name is Philipp and I"
generation_args = {'do_sample': True, 'max_length': 100}

# 调用measure_latency函数并输出结果
latency_info, p95_latency = measure_latency(model, tokenizer, payload, generation_args)
print(latency_info)


解码策略与生成设置
使用贪婪搜索（greedy search）作为解码策略，并将生成128个新的token，输入的长度也为128个token。

贪婪搜索（Greedy Search）：

贪婪搜索是一种常见的解码策略，在生成文本时，每一步选择概率最高的下一个token。虽然这种方法能够快速生成文本，但容易陷入局部最优解，可能导致生成的文本缺乏多样性。
与其他解码策略（如束搜索、采样方法）相比，贪婪搜索通常更快，但可能不如其他方法生成的文本质量高，尤其是在需要生成更复杂或更具创意的内容时。
生成设置：
在推理过程中，将模型的输入设置为128个token，这意味着输入文本将被分割为128个token的序列。
然后，模型将基于输入生成新的文本，生成的输出长度也为128个token。目的是确保生成的输出具有足够的上下文，以便进行有效的性能评估。


In [7]:
# 扩展输入文本（payload），通过重复两次相同的内容来增加输入序列的长度
payload = (
    "Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. "
    "What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer "
    "this email in the next 7 days. Best regards and have a nice weekend but it"
) * 2  # 将输入文本重复两次以扩展长度

# 打印扩展后的输入序列长度
print(f'Payload sequence length is: {len(tokenizer(payload)["input_ids"])}')

# 生成的参数设置
generation_args = dict(
    do_sample=False,  # 不使用采样，使用贪婪搜索
    num_beams=1,      # 使用单束搜索（不进行束搜索优化）
    min_length=128,   # 生成的最小长度为128个token
    max_new_tokens=128  # 最大生成128个新的token
)

# 使用测量延迟函数来评估未优化模型（Vanilla model）的延迟
vanilla_results = measure_latency(model, tokenizer, payload, generation_args)

# 打印未优化模型的延迟结果
print(f"Vanilla model: {vanilla_results[0]}")


Payload sequence length is: 128
Vanilla model: P95 latency (ms) - 8985.898722249989; Average latency (ms) - 8955.07 +\- 24.34;


模型在生成128个token的情况下，达到了8.9秒的推理延迟，相当于每个token的生成时间为69毫秒

## 3. 使用 DeepSpeeds 的 `InferenceEngine` 优化 GPT-J

- 接下来也是最重要的一步是优化模型以在GPU上进行推理。
- 使用DeepSpeed的InferenceEngine来实现。
- InferenceEngine通过init_inference方法进行初始化。init_inference方法至少需要以下几个参数：

- model: 需要优化的模型。
- mp_size: 使用的GPU数量（模型并行的数量）。
- dtype: 使用的数据类型（如float16）。
- replace_with_kernel_inject: 是否注入自定义CUDA内核。


In [ ]:
import torch  # 导入PyTorch库
from transformers import AutoTokenizer, AutoModelForCausalLM  # 导入Transformers库中的模型和分词器加载方法
import deepspeed  # 导入DeepSpeed库

# Hugging Face模型库中的模型仓库ID
model_id = "philschmid/gpt-j-6B-fp16-sharded"

# 加载模型和分词器
# 使用AutoTokenizer加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 使用AutoModelForCausalLM加载因果语言模型，设置权重数据类型为float16
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

# 初始化DeepSpeed推理引擎
# 参数解释：
# - model: 要优化的Transformer模型实例
# - mp_size: 使用的GPU数量（这里设置为1）
# - dtype: 模型权重的数据类型（这里设置为float16）
# - replace_method: 设置为"auto"，让DeepSpeed自动识别需要替换的层
# - replace_with_kernel_inject: 设置为True，使用DeepSpeed的内核注入器替换默认的CUDA内核
ds_model = deepspeed.init_inference(
    model=model,  # 需要优化的模型
    mp_size=1,  # 使用1个GPU
    dtype=torch.float16,  # 使用半精度浮点数（fp16）以减少内存占用
    replace_method="auto",  # 让DeepSpeed自动识别和替换需要优化的层
    replace_with_kernel_inject=True  # 使用DeepSpeed的内核注入器进行优化
)

# 打印确认模型已加载到哪个设备上
print(f"model is loaded on device {ds_model.module.device}")


现在检查模型的计算图，验证原始的GPTJLayer已经被HFGPTJLayer替换，而HFGPTJLayer包含了DeepSpeedTransformerInference模块。

```python
InferenceEngine(
  (module): GPTJForCausalLM(  # GPT-J的因果语言模型模块
    (transformer): GPTJModel(  # GPT-J的Transformer模型
      (wte): Embedding(50400, 4096)  # 词嵌入层（Embedding layer）
      (drop): Dropout(p=0.0, inplace=False)  # Dropout层，丢弃概率为0.0
      (h): ModuleList(  # 模型的主体部分是一个模块列表（包括多个Transformer层）
        (0): DeepSpeedTransformerInference(  # 使用DeepSpeed优化的Transformer推理模块
          (attention): DeepSpeedSelfAttention()  # 使用DeepSpeed优化的自注意力层
          (mlp): DeepSpeedMLP()  # 使用DeepSpeed优化的多层感知机（MLP）层
        )

```

In [9]:
from deepspeed.ops.transformer.inference import DeepSpeedTransformerInference  # 导入DeepSpeed的推理优化模块

# 断言检查：验证模型的第一个Transformer层是否是DeepSpeed优化的Transformer推理模块
assert isinstance(ds_model.module.transformer.h[0], DeepSpeedTransformerInference) == True, "Model not successfully initialized"


In [10]:
# 测试模型生成能力
# 定义一个简单的输入示例
example = "My name is Philipp and I"

# 使用分词器将输入示例转换为模型可接受的输入ID，并将其转移到模型所在的设备（GPU）
input_ids = tokenizer(example, return_tensors="pt").input_ids.to(model.device)

# 使用DeepSpeed优化后的模型生成文本
# 参数解释：
# - `do_sample=True`: 表示使用采样策略来生成输出，增加多样性
# - `max_length=100`: 生成的文本最大长度为100个token
logits = ds_model.generate(input_ids, do_sample=True, max_length=100)

# 将生成的token ID转换为人类可读的文本
output_text = tokenizer.decode(logits[0].tolist())

# 打印生成的文本结果
print(output_text)


'My name is Philipp and I live in Freiburg in Germany and I have a project called Cenapen. After three months in development already it is finally finished – and it is a Linux based device / operating system on an ARM Cortex A9 processor on a Raspberry Pi.\n\nAt the moment it offers the possibility to store data locally, it can retrieve data from a local, networked or web based Sqlite database (I’m writing this tutorial while I’'

## 4. 评价效率和速度

作为最后一步，需要详细分析优化后的模型性能。应用优化技术（如图优化和混合精度）不仅会影响性能（延迟），还可能对模型的准确性产生影响。因此，加速模型往往伴随着一定的权衡。

- 性能与准确性的权衡：

- 通过图优化（graph optimizations）和混合精度（mixed-precision）等技术，可以显著提高模型推理的速度和降低延迟。
但是，这些技术也可能导致模型的准确性有所下降。对于实际应用，优化的目标需要在性能和准确性之间找到合适的平衡点。

- 测试优化后的模型性能：

- 使用和原始模型（vanilla model）相同的生成参数（generation_args）来测试优化后的模型性能。这确保了测试的公平性，可以直接对比优化前后的性能差异。

In [11]:
# 扩展输入文本（payload），通过重复两次相同的内容来增加输入序列的长度
payload = (
    "Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. "
    "What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer "
    "this email in the next 7 days. Best regards and have a nice weekend but it"
    * 2  # 将输入文本重复两次以扩展长度
)

# 打印扩展后的输入序列长度
print(f'Payload sequence length is: {len(tokenizer(payload)["input_ids"])}')

# 生成参数设置
generation_args = dict(
    do_sample=False,  # 不使用采样，使用贪婪搜索
    num_beams=1,      # 使用单束搜索（不进行束搜索优化）
    min_length=128,   # 生成的最小长度为128个token
    max_new_tokens=128  # 最大生成128个新的token
)

# 使用之前定义的measure_latency函数来测试DeepSpeed优化后的模型的推理延迟
# 参数包括：DeepSpeed优化后的模型、分词器、扩展后的输入文本、生成参数以及模型所在的设备
ds_results = measure_latency(ds_model, tokenizer, payload, generation_args, ds_model.module.device)

# 打印DeepSpeed优化后的模型的延迟结果
print(f"DeepSpeed model: {ds_results[0]}")


Payload sequence length is: 128
DeepSpeed model: P95 latency (ms) - 6577.044982599967; Average latency (ms) - 6569.11 +\- 6.57;


In [12]:
# 定义输入文本（payload）
payload = (
    "Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. "
    "What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer "
    "this email in the next 7 days. Best regards and have a nice weekend but it"
)

# 使用分词器将输入文本转换为模型可接受的输入ID，并将其放置到与模型相同的设备（GPU）上
input_ids = tokenizer(payload, return_tensors="pt").input_ids.to(model.device)

# 打印输入的payload内容
print(f"input payload: \n\n{payload}")

# 使用DeepSpeed优化后的模型生成文本
# 参数解释：
# - `do_sample=False`: 不使用采样，使用确定性的方法生成文本
# - `num_beams=2`: 使用束宽为2的束搜索（Beam Search）来生成输出
# - `min_length=64`: 生成的最小长度为64个token
# - `max_new_tokens=64`: 最大生成64个新的token
logits = ds_model.generate(input_ids, do_sample=False, num_beams=2, min_length=64, max_new_tokens=64)

# 将生成的token ID转换为人类可读的文本
output_text = tokenizer.decode(logits[0].tolist()[len(input_ids[0]):])

# 打印模型生成的预测输出
print(f"prediction: \n\n{output_text}")


input payload: 
 
Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer this email in the next 7 days. Best regards and have a nice weekend but it
prediction: 
 
 's not over yet.

I am getting in touch with you because i didn't get a response from you. What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer this email in the next 7 days. Best regards and have a nice weekend but


优化后的DeepSpeed模型在生成128个token时达到了6.5秒的推理延迟，相当于每个token的生成时间为50毫秒。

性能提升分析
性能改进结果：

优化前的GPT-J-6B模型生成128个token的延迟为8.9秒（即69毫秒/token）。
优化后的模型生成128个token的延迟降低到6.5秒（即50毫秒/token）。
提升幅度计算：

优化前的每个token延迟为69毫秒，优化后为50毫秒。
计算性能提升倍数：
提升倍数1.38
结果表明，经过优化后，模型的推理速度提升了1.38倍。